# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [1]:
import networkx
homer = open('homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

Next implement a function to read in the edges from the input file.

In [3]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A list of the nodes in the graph of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function        
    lines = gfile.readlines()
    gfile.seek(0)
    chapters = extract_chapters(lines)
    characters = extract_characters(lines, chapters)
    return characters


def extract_chapters(lines):
    """Reads in the chapters from the input file
    
    Args:
        lines: the read lines of the gfile
        
    Returns:
        A list of the chapters in the book
    """
    chapters = []
    for line in lines:
        if line[0:2] == '\n':
            pass
        elif line[1] == ':':
            chapters.append(line[0:2])
        elif line[2] == ':':
            chapters.append(line[0:3])
    return list(set(chapters))


def extract_characters(lines, chapters):
    """Reads in the characters from the input file and chapters list
    
    Args:
        lines: the read lines of the gfile
        chapters: list of chapters which contain characters in book
        
    Returns:
        A list of the characters in the book
    """
    characters = []
    for line in lines:
        if any(line.startswith(x) for x in chapters):
            if line[1] == ':':
                add = line[2:len(line)-1].replace(';', ',').split(',')
                characters.append(add)
            else:
                add = line[3:len(line)-1].replace(';', ',').split(',')
                characters.append(add)
    return list(set([char for sublist in characters for char in sublist]))


In [4]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        The edges in the graph as a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    lines = gfile.readlines()
    gfile.seek(0)
    chapters = extract_chapters(lines)
    interactions = extract_interactions(lines, chapters)
    edges = extract_edges(interactions)
    return edges
        
def extract_interactions(lines, chapters):
    """Reads in the interations from the input file and chapters list
    
    Args:
        lines: the read lines of the gfile
        chapters: list of chapters which contain characters in book
        
    Returns:
        A list of the interactions in the book
    """
    interactions = []
    for line in lines:
        if any(line.startswith(x) for x in chapters):
            if line[1] == ':':
                add = line[2:len(line)-1].split(';')
                interactions.append(add)
            else:
                add = line[3:len(line)-1].split(';')
                interactions.append(add)
    return interactions

def extract_edges(interactions):
    """Creates list with each entry a tuple with the characters who interacted
    
    Args:
        interactions: a list of each set of interactions in the book
        
    Returns:
        A list interaction pairs (edges)
    """
    pair_edges = []
    for interaction_chapter in interactions:
        for interaction in interaction_chapter:
            pair_interactions = interaction.split(',')
            for i in pair_interactions:
                for j in pair_interactions:
                    if i !=j:
                        if (j,i) not in pair_edges:
                            pair_edges.append((i,j))
    return pair_edges

The following code should now correctly create the graph.

In [5]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [6]:
def Search(graph, root):
    """Runs Search from vertex root in a graph. Neighboring nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    nodes_visited = initialise(graph)
    path = DFS(graph, root, nodes_visited)
    return path


def deterministic_sort(node_list):
    """Sorts nodes in numeric-alphabet order
    
    Args:
        node_list: list of nodes to sort
        
    Returns:
        A list of nodes in the order in which they should be visited
    """
    numeric = []
    alphanumeric = []
    alphabet = []
    for i in node_list:
        if i.isnumeric():
            numeric.append(i)
        elif i[0].isnumeric():
            alphanumeric.append(i)
        else:
            alphabet.append(i)
    numeric.sort()
    alphanumeric.sort()
    alphabet.sort()
    return numeric + alphanumeric + alphabet


def initialise(graph):
    """Initialises tracker for nodes visited
    
    Args:
        graph: the graph input
        
    Returns:
        A dictionary with False recorded for every node as none have been visited yet
    """
    nodes = list(graph.nodes)
    nodes = deterministic_sort(nodes)
    nodes_visited = {}
    for node in nodes:
        nodes_visited[node] = False
    return nodes_visited
    
def DFS(graph, root, nodes_visited):
    """Runs DFS from the root node going only to nodes not already visited in sorted order
    
    Args:
        graph: the graph input
        root: the starting node
        nodes_visited: dictionary of nodes already visited
        
    Returns:
        A path from the root node to all nodes that are reachable
    """
    nodes_visited[root] = True
    path = []
    path.append(root)
    sorted_neighbors = deterministic_sort(list(graph.neighbors(root)))
    for neighbor in sorted_neighbors:
        if not(nodes_visited[neighbor]):
            nodes_visited[neighbor] = True
            path_recursive = DFS(graph, neighbor, nodes_visited)
            for node in path_recursive:
                path.append(node)
    return path
    

We will check the correctness of your code by verifying that it correctly computes the connected component of node Ulysses (node `OD`).

In [7]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [8]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    connected_paths = []
    nodes_visited = initialise(graph)
    for node in nodes_visited:
        if nodes_visited[node] == False:
            nodes_visited[node]
            paths = Search(graph, node)
            for i in paths:
                nodes_visited[i] = True
            connected_paths.append(paths)
    return connected_paths

We will check correctness of your code by verifying that your output is identical to our solution.

In [9]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [10]:
component_sizes = [len(c) for c in character_interactions]
print("There are 12 connected components in the Iliad:{}".format(len(component_sizes) == 12))
print("The giant component has size 542:{}".format(max(component_sizes) == 542))
print("There are 5 isolated characters:{}".format(len([c for c in component_sizes if c == 1]) == 5))

There are 12 connected components in the Iliad:True
The giant component has size 542:True
There are 5 isolated characters:True
